<a href="https://colab.research.google.com/github/TioHalu/Forest_Fire_Prediction_Colab/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
df=pd.read_csv('/content/drive/MyDrive/tugas/dataset/weatherHistory.csv')

In [14]:
#data preprocessing
col = ['Formatted Date','Summary','Precip Type','Apparent Temperature (C)','Humidity','Wind Speed (km/h)','Wind Bearing (degrees)','Visibility (km)','Loud Cover','Pressure (millibars)','Daily Summary']
data = df.drop(col, axis="columns", inplace=True)
data= df.drop(df.index[100:])
dataReal = df.drop(df.index[125:])

In [15]:

# Menormalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data['Temperature (C)'].values.reshape(-1,1))

# Membagi data menjadi data training dan data testing
train_size = int(len(data_scaled) * 0.7)
test_size = len(data_scaled) - train_size
train, test = data_scaled[0:train_size,:], data_scaled[train_size:len(data_scaled),:]

# Membuat data training dan testing dalam bentuk time series
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 7
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

# Membuat model LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

# Melakukan prediksi menggunakan data testing
Y_pred = model.predict(X_test)

# Mengubah kembali data yang sudah dinormalisasi menjadi data awal
Y_test = scaler.inverse_transform(Y_test.reshape(-1, 1))
Y_pred = scaler.inverse_transform(Y_pred.reshape(-1, 1))

# Menghitung evaluasi model
mse = mean_squared_error(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
mape = np.mean(np.abs((Y_test - Y_pred) / Y_test)) * 100
r2 = r2_score(Y_test, Y_pred)

# Menampilkan hasil evaluasi model
print('Mean Squared Error (MSE):', mse)
print('Mean Absolute Error (MAE):', mae)
print('Mean Absolute Percentage Error (MAPE):', mape)
print('R-Squared:', r2)

Epoch 1/100
62/62 - 2s - loss: 0.1300 - 2s/epoch - 35ms/step
Epoch 2/100
62/62 - 0s - loss: 0.0579 - 198ms/epoch - 3ms/step
Epoch 3/100
62/62 - 0s - loss: 0.0545 - 190ms/epoch - 3ms/step
Epoch 4/100
62/62 - 0s - loss: 0.0446 - 178ms/epoch - 3ms/step
Epoch 5/100
62/62 - 0s - loss: 0.0374 - 177ms/epoch - 3ms/step
Epoch 6/100
62/62 - 0s - loss: 0.0314 - 184ms/epoch - 3ms/step
Epoch 7/100
62/62 - 0s - loss: 0.0243 - 179ms/epoch - 3ms/step
Epoch 8/100
62/62 - 0s - loss: 0.0182 - 189ms/epoch - 3ms/step
Epoch 9/100
62/62 - 0s - loss: 0.0167 - 175ms/epoch - 3ms/step
Epoch 10/100
62/62 - 0s - loss: 0.0176 - 179ms/epoch - 3ms/step
Epoch 11/100
62/62 - 0s - loss: 0.0132 - 181ms/epoch - 3ms/step
Epoch 12/100
62/62 - 0s - loss: 0.0135 - 185ms/epoch - 3ms/step
Epoch 13/100
62/62 - 0s - loss: 0.0153 - 186ms/epoch - 3ms/step
Epoch 14/100
62/62 - 0s - loss: 0.0160 - 184ms/epoch - 3ms/step
Epoch 15/100
62/62 - 0s - loss: 0.0111 - 180ms/epoch - 3ms/step
Epoch 16/100
62/62 - 0s - loss: 0.0110 - 181ms/epoc